# Classificador de draft da NBA
O objetivo do modelo é, a partir dos dados das temporadas 1996-97 a 2021-22, classificar corretamente se o jogador é *undrafted*, ou seja, não foi draftado.

Passo a passo:
* Escolha do problema ✔
* Separação dos dados em treino, validação e teste ✔
* Selecionar algoritmos para resolver o problema 
* Adicionar MLFlow no treinamento dos modelos
* Executar uma ferramenta de seleção de hiper-parâmetros
* Realizar diagnóstico do melhor modelo e melhorá-lo a partir disso

## Dependências

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Path Tales
path = '/content/drive/MyDrive/2022.1/TA GDI/projeto1/data/classification.csv'

In [4]:
dataset = pd.read_csv(path)

In [5]:
dataset.head()

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,undrafted,season_start_year,season_end_year,gp_pct
0,2071,16,0.153846,0.750000,0.485714,65,72,0.568966,0.125,0.175758,...,0.127,0.182,0.142,0.536,0.052,0,0.0,0.0,0.0,0.865854
1,1475,13,0.461538,0.678571,0.485714,117,72,NaN,NaN,NaN,...,0.016,0.115,0.151,0.535,0.099,0,1.0,0.0,0.0,0.865854
2,1466,2,0.423077,0.714286,0.533333,227,72,NaN,NaN,NaN,...,0.083,0.152,0.167,0.542,0.101,0,1.0,0.0,0.0,0.902439
3,1465,9,0.153846,0.642857,0.485714,189,72,0.568966,0.125,0.151515,...,0.109,0.118,0.233,0.482,0.114,0,0.0,0.0,0.0,0.512195
4,1464,35,0.153846,0.535714,0.438095,249,72,0.551724,0.250,0.303030,...,0.087,0.045,0.135,0.470,0.125,0,0.0,0.0,0.0,0.109756


In [12]:
dataset.groupby(['undrafted'])['player_name'].count()

undrafted
0.0    9629
1.0    1875
Name: player_name, dtype: int64

## Separação dos conjuntos de treino, validação e teste

In [10]:
RANDOM_STATE = 42

In [6]:
dataset.columns

Index(['player_name', 'team_abbreviation', 'age', 'player_height',
       'player_weight', 'college', 'country', 'draft_year', 'draft_round',
       'draft_number', 'gp', 'pts', 'reb', 'ast', 'net_rating', 'oreb_pct',
       'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct', 'season', 'undrafted',
       'season_start_year', 'season_end_year', 'gp_pct'],
      dtype='object')

Escolhemos as colunas de altura, peso, faculdade, país, jogos realizados e métricas de desempenho (pts, reb, ast...) para compor o conjunto de features e a coluna 'undrafted' é nossa label

In [8]:
X = dataset[['player_height', 'player_weight', 'college', 'country', 
             'gp', 'pts', 'reb', 'ast', 'net_rating', 'oreb_pct',
             'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct']]
y = dataset['undrafted']
X.shape, y.shape

((11504, 14), (11504,))

Designamos 70% dos dados para teste, 20% para treino e 10% para validação

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=RANDOM_STATE)

X_train.shape, X_val.shape, X_test.shape

((8052, 14), (1151, 14), (2301, 14))

## Seleção de algoritmos
Selecionamos os seguintes para resolução do problema de classificação:
* MLP
* RandomForest
* XGBoost
* LogisticRegression
Além disso, como notamos que o dataset é bem desbalanceado, utilizaremos o f1 score como métrica de treinamento.

### MLPs

In [14]:
EPOCHS=500

#### Default

Realizando o treino do modelo

In [15]:
mlp = MLPClassifier(random_state=RANDOM_STATE, max_iter=EPOCHS).fit(X_train, y_train)

Validando:

In [18]:
y_pred = mlp.predict(X_val)
f1_score(y_val, y_pred, average='weighted')

0.754230082735617

#### Tentativa 2


```
hidden_layers = (10,10)
```



In [21]:
mlp = MLPClassifier(random_state=RANDOM_STATE, max_iter=EPOCHS, hidden_layer_sizes=(10, 10)).fit(X_train, y_train)

Validando:

In [22]:
y_pred = mlp.predict(X_val)
f1_score(y_val, y_pred, average='weighted')

0.7431908403435481

#### Tentativa 3

```
hidden_layers = (10,10)
activation = 'logistic'
```




In [19]:
mlp = MLPClassifier(random_state=RANDOM_STATE, max_iter=EPOCHS, hidden_layer_sizes=(10, 10), activation='logistic').fit(X_train, y_train)

Validando:

In [20]:
y_pred = mlp.predict(X_val)
f1_score(y_val, y_pred, average='weighted')

0.7456095918283264